# Función de Perdida

Todos los algoritmos en Deep Learning dependen de minimizar alguna función, la *función objetivo*, que es conocida como la *función de perdida*. Una función de perdida nos permite medir que tan bien está prediciendo el resultado esperado un modelo predictivo.